In [1]:
import requests
import time
import json
import selenium.webdriver.support.ui as ui
from selenium.webdriver.common.keys import Keys
import threading
import queue
import random
import datetime
import pyperclip

from urllib.error import URLError, HTTPError
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.remote.webelement import WebElement
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from pprint import pprint
from message_config import *

In [2]:
broadcast_id = []               # 방송 ID
broadcast_url = []              # 방송 URL
broadcast_starttime = []        # 방송 시작시간
broadcast_fulltime = []
bool_onair = []                 # 방송시청 판단 여부, True/False
bool_reward = []
broadcast_count = 0             # 방송 개수
tabs_num = []                   # Chrome Browser 관리


In [3]:
def load_broadcast_info():
    global broadcast_id
    global broadcast_url
    global broadcast_starttime
    global bool_onair
    global broadcast_count
    global broadcast_fulltime

    
    broadcast_id = []
    broadcast_url = []
    broadcast_starttime = []
    broadcast_fulltime = []
    bool_onair = []
    broadcast_count = 0
    broadcast_timestamp = 0


    
    # 오늘의 날짜, Day
    todays_date = datetime.datetime.today().strftime('%d')

    response_milestone = requests.get('https://apis.naver.com/selectiveweb/live_commerce_web/v2/broadcast/milestones')
    dict_milestone = json.loads(response_milestone.text)


    for _milestone in dict_milestone:
        if int(todays_date) == _milestone['milestone']['date']:
            broadcast_timestamp = _milestone['milestone']['timestamp']  # 해당 날짜 타임스템프
            broadcast_count = _milestone['broadcastCount']

    response_dailyshow = requests.get(f'https://apis.naver.com/selectiveweb/selectiveweb/v1/lives/timeline/daily?next={broadcast_timestamp}&size=100')
    dict_dailyshow = json.loads(response_dailyshow.text)

    for _idx, _dailyshow in enumerate(dict_dailyshow['list']):
        broadcast_id.append(_dailyshow['broadcastId'])
        broadcast_url.append(_dailyshow['broadcastEndUrl'])
        broadcast_starttime.append(int(_dailyshow['expectedStartDate'][11:13] + _dailyshow['expectedStartDate'][14:16]))
        broadcast_fulltime.append(_dailyshow['expectedStartDate'])

    bool_onair = [False for _ in range(len(broadcast_id))]
    
    broadcast_fulltime = [new_text.replace('T', ' ') for new_text in broadcast_fulltime]
    broadcast_fulltime = [datetime.datetime.strptime(new_type[0:19], '%Y-%m-%d %H:%M:%S') for new_type in broadcast_fulltime]

    print("--------------------------------------------------")
    print(datetime.datetime.today().strftime(f'%Y년 %m월 %d일'))
    print(f'방송 갯수: {broadcast_count}')
    print(f'방송 ID: {broadcast_id}')
    print(f'방송 시간: {broadcast_starttime}')
    print(f'방송 상태: {bool_onair}')
    print("--------------------------------------------------")

In [4]:
def broadcast_connection(_idx):
    global tabs_num
    lock.acquire()
    tabs = driver.window_handles
    driver.switch_to.window(tabs[tabs_num.index(_idx)])
    driver.set_window_position(random.randrange(1600), random.randrange(800))
    driver.get(broadcast_url[_idx])
    lock.release()

In [5]:
def comment_button_click(_idx):
    global tabs_num
    lock.acquire()
    tabs = driver.window_handles
    driver.switch_to.window(tabs[tabs_num.index(_idx)])
    try:
        driver.find_element_by_class_name('CommentButton_icon_3iWZs').click()
    except:
        print(f">> ERROR: {_idx}, Comment1, 버튼 클릭")
    lock.release()

In [6]:
def input_comment(_idx, _step):
    global tabs_num
    lock.acquire()
    tabs = driver.window_handles
    driver.switch_to.window(tabs[tabs_num.index(_idx)])
    try:
        driver.find_element_by_xpath('//*[@id="wa_textarea"]').send_keys(list_comments[random.randrange(len(list_comments))])
    except:
        print(f">> ERROR: {_idx}, Comment{_step}, 코멘트 입력")
    lock.release()


In [7]:
def comment_enter(_idx, _step):
    global tabs_num
    lock.acquire()
    tabs = driver.window_handles
    driver.switch_to.window(tabs[tabs_num.index(_idx)])
    try:
        #driver.find_element_by_class_name('CommentWrite_button_2pJxH').click()
        driver.find_element_by_xpath('//*[@id="wa_textarea"]').send_keys(Keys.ENTER)
    except:
        print(f">> ERROR: {_idx}, Comment1, 엔터")
    lock.release()

In [8]:
def close_browser(_idx):
    global tabs_num
    global broadcast_starttime

    lock.acquire()
    tabs = driver.window_handles
    driver.switch_to.window(tabs[tabs_num.index(_idx)])
    driver.close()
    tabs_num.remove(_idx)
    try:
        exit_time = datetime.datetime.now().strftime(f'%H:%M:%S')
    except:
        print(f">> ERROR: {_idx}, 창닫기")
    lock.release()

    print(f"{exit_time} >> {_idx}, {broadcast_starttime[_idx]}, Broadcast Close!")


In [9]:
def copy_input(xpath, input):
    pyperclip.copy(input)
    driver.find_element_by_xpath(xpath).click()
    ActionChains(driver).key_down(Keys.CONTROL).send_keys('v').key_up(Keys.CONTROL).perform()
    time.sleep(1)

In [10]:
# ==================================================================================
# Thread Start
# ==================================================================================

def thread_onair(onair_idx, onair_time, start_time):
    global tabs_num
    bool_run_onair = True
    
    # Open the browser & connect to broadcast
    tabs = driver.window_handles
    if len(tabs) == 1:
        driver.switch_to.window(tabs[0])
    driver.execute_script('window.open("/", "", "_blank");')

    lock.acquire()
    tabs_num.append(onair_idx)
    lock.release()
    broadcast_connection(onair_idx)
    
    # gap_time = 0
    while(bool_run_onair):
        gap_time = (datetime.datetime.now() - start_time).seconds
        
        # Comment 1
        if gap_time == 600:     # 10분 뒤 (방송시작 5분 후)
            comment_button_click(onair_idx)        # 버튼 클릭
            time.sleep(0.6)
            input_comment(onair_idx, 1)            # 코멘트 입력
            time.sleep(0.5)
            comment_enter(onair_idx, 1)               # 엔터

        # Comment 2
        if gap_time == 900:    # 15분 뒤 (방송시작 10분 후)
            input_comment(onair_idx, 2)            # 코멘트 입력
            time.sleep(0.6)
            comment_enter(onair_idx, 2)               # 엔터
            time.sleep(0.6)

        if gap_time == 4200: # 70분 뒤 (방송시작 65분 후)
            close_browser(onair_idx)
            bool_run_onair = False


In [12]:
# # ==================================================================================
# # Main function
# # ==================================================================================
# id = 'id'
# pw = 'pw'

# if __name__ == "__main__":
#     load_broadcast_info()

#     DRIVER_PATH = './chromedriver'
#     CHROME_OPTIONS = Options()
#     CHROME_OPTIONS.add_argument("--window-size=200,150")
#     driver = webdriver.Chrome(executable_path=DRIVER_PATH, chrome_options=CHROME_OPTIONS)

#     # Login
#     tabs = driver.window_handles
#     driver.switch_to.window(tabs[0])
#     #driver.get('http://www.naver.com/')
    
#     driver.get('https://nid.naver.com/nidlogin.login?mode=form&url=https%3A%2F%2Fwww.naver.com')
#     copy_input('//*[@id="id"]', id)
#     time.sleep(1)
#     copy_input('//*[@id="pw"]', pw)
#     time.sleep(1)
#     driver.find_element_by_xpath('//*[@id="login_keep_wrap"]/div[1]/label').click()
#     time.sleep(1)
#     driver.find_element_by_xpath('//*[@id="log.login"]/span').click()
    
#     time.sleep(15)
#     tabs_num.append(9999)
    
#     bool_main_refresh = False       # Main browser refresh
#     bool_new_day = False

#     refresh_time = [10, 23, 40, 53]

#     lock = threading.Lock()

#     random.shuffle(list_comments)
    
#     while(1):
#         nowtime = datetime.datetime.now()
#         nowtime_hour_min = datetime.datetime.now().strftime('%H%M')

#         # -----------------------------------
#         # Main browser auto refresh
#         # -----------------------------------
#         if (int(nowtime_hour_min[2:4]) in refresh_time) and not bool_main_refresh:
#             lock.acquire()
#             print(f"{nowtime.strftime(f'%H:%M:%S')} >> Main browser refresh")
#             tabs = driver.window_handles
#             driver.switch_to.window(tabs[0])
#             driver.refresh()
#             bool_main_refresh = True
#             lock.release()
#         elif (int(nowtime_hour_min[2:4])-1 in refresh_time) and bool_main_refresh:
#             lock.acquire()
#             bool_main_refresh = False
#             lock.release()

#         # -----------------------------------
#         # Update new day information (AM 09:45)
#         # -----------------------------------
#         if (nowtime.strftime('%H%M') == '0945') and not bool_new_day:
#             print(f"{nowtime.strftime(f'%H:%M:%S')} >> Update new day information")
#             load_broadcast_info()
#             bool_new_day = True
#         elif (nowtime.strftime('%H%M') == '0946') and bool_new_day:
#             bool_new_day = False

#         # -----------------------------------
#         # On air thread
#         # -----------------------------------
#         #nowtime = datetime.datetime(2020, 12, 23, 10, 56, 37, 20143)
#         nowtime = datetime.datetime.now()

#         for idx, _fulltime in enumerate(broadcast_fulltime):
#             if (_fulltime - nowtime).seconds < 300 and not bool_onair[idx]:     # 5분전 실행
#                 bool_onair[idx] = True
                
#                 # EX) 13:50:39 >> 11, 56241, 1400: Broadcast start!
#                 print(f"{nowtime.strftime(f'%H:%M:%S')} >> {idx}, {broadcast_id[idx]}, {broadcast_starttime[idx]}: Broadcast start!")
#                 onair_thread = threading.Thread(target=thread_onair, args=(idx, broadcast_starttime[idx], nowtime))
#                 onair_thread.start()
#                 time.sleep(1)
#                 break

#         time.sleep(0.1)



In [12]:
load_broadcast_info()

--------------------------------------------------
2022년 03월 16일
방송 갯수: 93
방송 ID: [447194, 432872, 437859, 449647, 450833, 443832, 432501, 420578, 447411, 447425, 442621, 443601, 436611, 417945, 446134, 443478, 449995, 431297, 439960, 444173, 445342, 450034, 443940, 438684, 446016, 429877, 449821, 446630, 445483, 447193, 429174, 449644, 442210, 445233, 441997, 442907, 445284, 445266, 446917, 449678, 446582, 447135, 446845, 445637, 434447, 449558, 449990, 447373, 445683, 441752, 447748, 443301, 441841, 446142, 445389, 439926, 446893, 437910, 445526, 438254, 440225, 443060, 443482, 446214, 447075, 438353, 449399, 445454, 436679, 431318, 445255, 442596, 445195, 445793, 420039, 445722, 447361, 443877, 417952, 445370, 443239, 446734, 442169, 445299, 442917, 445211, 423809, 440168, 445677, 445869, 441991, 443335, 440102]
방송 시간: [900, 1000, 1000, 1000, 1000, 1000, 1030, 1030, 1030, 1100, 1100, 1100, 1100, 1100, 1100, 1100, 1100, 1100, 1100, 1100, 1100, 1100, 1100, 1100, 1100, 1100, 1100, 1100

In [13]:
DRIVER_PATH = './chromedriver'
CHROME_OPTIONS = Options()
CHROME_OPTIONS.add_argument("--window-size=200,150")
driver = webdriver.Chrome(executable_path=DRIVER_PATH, chrome_options=CHROME_OPTIONS)


<ipython-input-13-fbea370fef7f>:4: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path=DRIVER_PATH, chrome_options=CHROME_OPTIONS)


In [14]:
# Login
tabs = driver.window_handles
driver.switch_to.window(tabs[0])
#driver.get('http://www.naver.com/')

In [15]:
driver.get('https://nid.naver.com/nidlogin.login?mode=form&url=https%3A%2F%2Fwww.naver.com')
copy_input('//*[@id="id"]', id)
time.sleep(0.5)
copy_input('//*[@id="pw"]', pw)
time.sleep(0.5)
driver.find_element_by_xpath('//*[@id="login_keep_wrap"]/div[1]/label').click()
time.sleep(0.5)
driver.find_element_by_xpath('//*[@id="log.login"]/span').click()

In [16]:
time.sleep(5)
tabs_num.append(9999)

In [17]:
tabs_num

[9999]

In [18]:
bool_main_refresh = False       # Main browser refresh
bool_new_day = False

In [19]:
refresh_time = [13, 37]

In [20]:
nowtime = datetime.datetime.now()
nowtime_hour_min = datetime.datetime.now().strftime('%H%M')

In [22]:
nowtime

datetime.datetime(2022, 3, 16, 10, 38, 48, 386625)

In [23]:
nowtime_hour_min

'1038'

In [24]:
nowtime = datetime.datetime(2022, 3, 16, 10, 29, 37, 20143)
#nowtime = datetime.datetime.now()

for idx, _fulltime in enumerate(broadcast_fulltime):
    if (_fulltime - nowtime).seconds < 300 and not bool_onair[idx]:     # 5분전 실행
        bool_onair[idx] = True

        # EX) 13:50:39 >> 11, 56241, 1400: Broadcast start!
        print(f"{nowtime.strftime(f'%H:%M:%S')} >> {idx}, {broadcast_id[idx]}, {broadcast_starttime[idx]}: Broadcast start!")
        onair_thread = threading.Thread(target=thread_onair, args=(idx, broadcast_starttime[idx], nowtime))
        onair_thread.start()
        time.sleep(1)
        break

Exception in thread Thread-6:
Traceback (most recent call last):
  File "C:\Users\wnsdh\anaconda3\envs\myproject\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "C:\Users\wnsdh\anaconda3\envs\myproject\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-10-679a2304b421>", line 15, in thread_onair
NameError: name 'lock' is not defined


10:29:37 >> 6, 432501, 1030: Broadcast start!


In [25]:
idx

6

In [ ]:

lock = threading.Lock()
random.shuffle(list_comments)
tictoc_second = 0

while(1):
    nowtime = datetime.datetime.now()    # datetime.datetime(2022, 3, 8, 20, 19, 13, 842122)
    nowtime_hour_min = datetime.datetime.now().strftime('%H%M')    # 2019

    # -----------------------------------
    # 메인 로그인 브라우저 새로고침
    # -----------------------------------
    if (int(nowtime_hour_min[2:4]) in refresh_time) and not bool_main_refresh:
        lock.acquire()
        print(f"{nowtime.strftime(f'%H:%M:%S')} >> Main browser refresh")
        tabs = driver.window_handles
        driver.switch_to.window(tabs[0])
        driver.refresh()
        bool_main_refresh = True
        lock.release()
    elif (int(nowtime_hour_min[2:4])-1 in refresh_time) and bool_main_refresh:
        lock.acquire()
        bool_main_refresh = False
        lock.release()

    # -----------------------------------
    # 당일 방송정보 업데이트 (AM 09:45)
    # -----------------------------------
    if (nowtime.strftime('%H%M') == '0945') and not bool_new_day:
        print(f"{nowtime.strftime(f'%H:%M:%S')} >> 새로운 방송정보 업데이트")
        load_broadcast_info()
        bool_new_day = True
    elif (nowtime.strftime('%H%M') == '0946') and bool_new_day:
        bool_new_day = False

    # -----------------------------------
    # 생방송 스레드
    # -----------------------------------
    #nowtime = datetime.datetime(2020, 12, 23, 10, 56, 37, 20143)
    nowtime = datetime.datetime.now()

    for idx, _fulltime in enumerate(broadcast_fulltime):
        if (_fulltime - nowtime).seconds < 300 and not bool_onair[idx]:     # 5분전 실행
            bool_onair[idx] = True

            # EX) 13:50:39 >> 11, 56241, 1400: Broadcast start!
            print(f"{nowtime.strftime(f'%H:%M:%S')} >> {idx}, {broadcast_id[idx]}, {broadcast_starttime[idx]}: Broadcast start!")
            onair_thread = threading.Thread(target=thread_onair, args=(idx, broadcast_starttime[idx], nowtime))
            onair_thread.start()
            time.sleep(1)
            break

    time.sleep(0.1)



--------------------------------------------------
2022년 03월 08일
방송 갯수: 78
방송 ID: [439778, 437206, 441890, 432982, 417929, 426649, 419813, 440226, 430616, 438043, 440086, 440018, 429703, 432586, 436451, 438379, 430244, 435212, 442626, 437439, 410837, 437407, 435065, 438077, 438483, 436405, 426555, 431641, 433448, 440833, 421111, 438471, 429937, 440002, 437112, 432793, 434814, 442370, 438882, 435401, 442365, 437223, 435304, 442282, 439826, 436751, 441185, 438773, 439851, 429764, 432975, 437430, 431325, 421235, 432415, 435773, 439041, 429862, 438478, 429723, 426975, 435259, 429733, 435252, 422311, 440396, 435370, 435302, 438495, 441177, 440238, 421216, 437481, 440511, 431703, 433468, 438859]
방송 시간: [800, 900, 930, 1000, 1000, 1000, 1000, 1010, 1030, 1030, 1030, 1100, 1100, 1100, 1100, 1100, 1100, 1100, 1100, 1100, 1100, 1100, 1100, 1100, 1100, 1100, 1100, 1100, 1200, 1200, 1200, 1230, 1230, 1300, 1400, 1400, 1400, 1400, 1400, 1500, 1600, 1600, 1700, 1700, 1700, 1800, 1800, 1800, 1800, 18

<ipython-input-12-1707bed8f193>:13: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path=DRIVER_PATH, chrome_options=CHROME_OPTIONS)


2115


In [168]:
nowtime = datetime.datetime.now()
nowtime

datetime.datetime(2022, 3, 8, 20, 32, 1, 889056)

In [169]:
broadcast_fulltime

[datetime.datetime(2022, 3, 8, 8, 0, 6),
 datetime.datetime(2022, 3, 8, 9, 0, 33),
 datetime.datetime(2022, 3, 8, 9, 30, 29),
 datetime.datetime(2022, 3, 8, 10, 0, 29),
 datetime.datetime(2022, 3, 8, 10, 0, 41),
 datetime.datetime(2022, 3, 8, 10, 0, 27),
 datetime.datetime(2022, 3, 8, 10, 0, 3),
 datetime.datetime(2022, 3, 8, 10, 10, 43),
 datetime.datetime(2022, 3, 8, 10, 30, 23),
 datetime.datetime(2022, 3, 8, 10, 30, 53),
 datetime.datetime(2022, 3, 8, 10, 30, 49),
 datetime.datetime(2022, 3, 8, 11, 0, 58),
 datetime.datetime(2022, 3, 8, 11, 0, 52),
 datetime.datetime(2022, 3, 8, 11, 0, 6),
 datetime.datetime(2022, 3, 8, 11, 0, 49),
 datetime.datetime(2022, 3, 8, 11, 0, 43),
 datetime.datetime(2022, 3, 8, 11, 0, 15),
 datetime.datetime(2022, 3, 8, 11, 0, 27),
 datetime.datetime(2022, 3, 8, 11, 0, 47),
 datetime.datetime(2022, 3, 8, 11, 0, 14),
 datetime.datetime(2022, 3, 8, 11, 0, 50),
 datetime.datetime(2022, 3, 8, 11, 0, 1),
 datetime.datetime(2022, 3, 8, 11, 0, 23),
 datetime.dat

In [170]:
datetime.datetime.now()

datetime.datetime(2022, 3, 8, 20, 33, 9, 536869)

In [ ]:
#nowtime = datetime.datetime(2020, 12, 23, 10, 56, 37, 20143)
nowtime = datetime.datetime.now()

for idx, _fulltime in enumerate(broadcast_fulltime):
    if (_fulltime - nowtime).seconds < 60 and not bool_onair[idx]:     # 1분전 실행, 실행중이 아닐때
        
        # 방송 리워드 체크
        if bool_reward == 0:
            outerHTML = requrests.get('https://view.shoppinglive.naver.com/lives/{}'.format(broadcast_id[idx]))
            useReward = 'useReward\\":true'
            bool_reward[idx] = outerHTML.text.find(useReward)
        
        # 방송 리워드 True
        if bool_reward[idx] > 0:
            bool_onair[idx] = True
            print(f"{nowtime.strftime(f'%H:%M:%S')} >> {idx}, {broadcast_id[idx]}, {broadcast_starttime[idx]}: Broadcast start!")
            onair_thread = threading.Thread(target=thread_onair, args=(idx, broadcast_starttime[idx], nowtime))
            onair_thread.start()
            time.sleep(1)
            break

In [178]:
outerHTML = requests.get('https://view.shoppinglive.naver.com/lives/437430')

print()

2115


'https://view.shoppinglive.naver.com/lives/439778'

In [184]:
time1 = datetime.datetime.now()
time1

datetime.datetime(2022, 3, 8, 21, 43, 18, 911642)

In [196]:
time2 = datetime.datetime.now()
time2

datetime.datetime(2022, 3, 8, 21, 46, 22, 856213)

In [197]:
(time2 - time1).seconds

183

In [ ]:
gap_time = (datetime.datetime.now() - start_time).seconds

In [4]:
try:
    try:
        a = '1'
        b = 1
        a + b
    except:
        print("22")
except:
    print("except")


22


TypeError: can only concatenate str (not "int") to str

In [13]:
lista = ['1234']

In [14]:
lista

['1234']

In [15]:
lista[0]

'1234'

In [16]:
print(f'{lista[0][0:2]}:{lista[0][2:4]}')

12:34


In [27]:
onair_idx = 6

bool_run_onair = True

# Open the browser & connect to broadcast
tabs = driver.window_handles
if len(tabs) == 1:
    driver.switch_to.window(tabs[0])
driver.execute_script('window.open("/", "", "_blank");')


tabs_num.append(onair_idx)

broadcast_connection(onair_idx)

NameError: name 'lock' is not defined

In [ ]:

def thread_onair(onair_idx, onair_time, start_time):

    
    # gap_time = 0
    while(bool_run_onair):
        gap_time = (datetime.datetime.now() - start_time).seconds
        
        # Comment 1
        if gap_time == 600:     # 10분 뒤 (방송시작 5분 후)
            comment_button_click(onair_idx)        # 버튼 클릭
            time.sleep(0.6)
            input_comment(onair_idx, 1)            # 코멘트 입력
            time.sleep(0.5)
            comment_enter(onair_idx, 1)               # 엔터

        # Comment 2
        if gap_time == 900:    # 15분 뒤 (방송시작 10분 후)
            input_comment(onair_idx, 2)            # 코멘트 입력
            time.sleep(0.6)
            comment_enter(onair_idx, 2)               # 엔터
            time.sleep(0.6)

        if gap_time == 4200: # 70분 뒤 (방송시작 65분 후)
            close_browser(onair_idx)
            bool_run_onair = False
